In [1]:
import pandas as pd
import numpy as np

class SME():
    
    def __init__(self):
        self.asked = 0
        self.df = self.get()
        
    def get(self):
        X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')
        y_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_y.csv')

        X_train['Left'] = y_train
        return X_train
    
    def ask(self, valuedict):
        self.asked += 1
        #  print(self.asked)
        
        if self.asked > 500000:
            raise Exception("Sorry, you have asked enough")

        arr = []
        for prop in valuedict:
            val = valuedict[prop]

            if val is None: 
                continue

            val = f"'{val}'" if isinstance(val,str) else val
            # print(prop, val)
            arr.append(f'{prop} == {val}')

            query = ' and '.join(arr)
        result = self.df.query(query)

        if len(result) == 0:
            raise Exception("I don't know")
        
        return (int)(result['Left'].mean())

In [2]:
sme = SME()

In [3]:
sme.df.columns

Index(['RecordId', 'Gender', 'Distance', 'YearsWorked', 'TrainingHours',
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'DepartmentCode', 'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Left'],
      dtype='object')

In [4]:
X_train = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/final/employee_departure_dataset_X.csv')

In [5]:
sme.df['SelfReview'] = sme.df['SelfReview'].fillna(sme.df['SelfReview'].median())
sme.df['SupervisorReview'] = sme.df['SupervisorReview'].fillna(sme.df['SupervisorReview'].median())
sme.df[['Salary', 'PreviousSalary']] = sme.df[['Salary', 'PreviousSalary']].map(lambda x: x.replace('K', '000') if isinstance(x, str) else x)
sme.df[['Salary', 'PreviousSalary']] = sme.df[['Salary', 'PreviousSalary']].astype(int)

sme.df['Raise'] = (sme.df['Salary'] / sme.df['PreviousSalary']) - 1
sme.df['ReviewDiff'] = sme.df['SelfReview'] - sme.df['SupervisorReview']
sme.df['ReviewOverRaise'] = sme.df['SelfReview'] / (sme.df['Raise'] +.0001)

sme.df['LongDist'] = np.where(sme.df['Distance'].isin(['~20miles', '>30miles']), 1, 0)

sme.df['HighStress'] = np.where(sme.df['StressLevel'] > 3.5, 1, 0)

num = ['HighStress', 'LongDist','Gender', 'SupervisorReview','SelfReview', 'Raise', 'Salary','ReviewOverRaise'] #'ReviewOverRaise','ReviewDiff', 'StressLevel'
cat = ['DepartmentCode']#'Distance',
#num = ['Raise', 'StressLevel']

"""
num = ['HighStress', 'LongDist','Gender','RecordId', 'YearsWorked', 'TrainingHours', 
       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',
       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',
       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',
       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',
       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',
       'PreviousSalary', 'Salary', 'SelfReview',
       'SupervisorReview', 'Raise', 'ReviewDiff', 'ReviewOverRaise']
"""

"\nnum = ['HighStress', 'LongDist','Gender','RecordId', 'YearsWorked', 'TrainingHours', \n       'WorkLifeBalance', 'NumOfProjects', 'JobInvolvement', 'TeamSize',\n       'MentorshipReceived', 'TechSkillLevel', 'AttendanceRate', 'StressLevel',\n       'PeerFeedbackScore', 'AnnualLeaveDays', 'Certifications',\n       'SkillDevelopmentCourses', 'ProjectComplexity', 'WorkSatisfactionScore',\n       'JobEngagementScore', 'PhysicalActivityScore', 'MentalWellbeingScore',\n       'PreviousSalary', 'Salary', 'SelfReview',\n       'SupervisorReview', 'Raise', 'ReviewDiff', 'ReviewOverRaise']\n"

# Establish the preprocessing for numeric and categorical features.

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans

n_clusters = 500

featureprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num)#,
        #('cat', OneHotEncoder(), cat)
    ])
preprocessor = Pipeline(steps=[
    ('features', featureprocessor),
    ('pca', PCA(n_components=0.9)),
    ('kmeans', KMeans(n_clusters=n_clusters, random_state=42, n_init='auto'))
])
#('kmeans', KMeans(n_clusters=n_clusters, n_init='auto'), num)#+cat)
#('kmeans', KMeans(n_clusters=n_clusters, n_init='auto')

In [8]:
X_dist = preprocessor.fit_transform(sme.df[num+cat])

In [9]:
preprocessor

Pipeline(steps=[('features',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['HighStress', 'LongDist',
                                                   'Gender', 'SupervisorReview',
                                                   'SelfReview', 'Raise',
                                                   'Salary',
                                                   'ReviewOverRaise'])])),
                ('pca', PCA(n_components=0.9)),
                ('kmeans',
                 KMeans(n_clusters=500, n_init='auto', random_state=42))])

In [10]:
kmeans = KMeans(n_clusters = n_clusters, n_init='auto')
pre = featureprocessor.fit_transform(sme.df[num+cat])

In [11]:
X_dist = kmeans.fit_transform(pre)
X_dist

array([[4.27075289, 2.18906185, 3.73032674, ..., 3.04856675, 4.16362393,
        3.43010667],
       [2.15930482, 3.19133776, 2.21416168, ..., 3.58987003, 1.68254697,
        3.27334457],
       [4.30078498, 2.9491818 , 3.12369086, ..., 4.22292685, 4.20310753,
        4.61616773],
       ...,
       [4.85006033, 4.52845628, 3.6071235 , ..., 5.6971264 , 5.12726554,
        4.43018188],
       [2.05756492, 3.1039046 , 2.26426998, ..., 3.67810165, 2.54300597,
        3.49210281],
       [4.0037526 , 2.90216922, 4.0882788 , ..., 3.85635078, 3.99717466,
        3.97002199]])

In [12]:
X_dist.shape

(500000, 500)

In [13]:
#How many points are in each cluster?
pd.Series(kmeans.labels_).value_counts()

257    3729
67     3434
22     3347
475    3154
112    2971
       ... 
274     209
368     180
496     178
188     155
208     139
Name: count, Length: 500, dtype: int64

In [14]:
import numpy as np
#What are the reporesentative points? (e.i the ones closest to each centroid)
representative_idx = np.argmin(X_dist, axis=0)
representative_idx.shape

(500,)

In [15]:
#What is the index number of each representative point?
representative_idx

array([ 10976,    743,   2789,   7042,   8579,   2807,  10851,   1402,
         2829,   9517,    713,   1561,   9036,    585,   5950,  12000,
          779,   4848,  11039,  59193,   6937,   4150,   2538,  10963,
         5600,   1520, 108864,  28816,  20238,    428,   3484,  12558,
        24062,  24804,  12383,   3045,   5381,  39616,    815, 137007,
         2732,   5380,   3360,  20169,   5129,   9702,  11724,   5748,
        71581,   1283,   5002,   5389,  44945,   1426,  30165,   2275,
        30797,   5736,  46705,   2216,    139,      1,   7864,   7576,
         3187,   6138,  18073,   1082,  91763,   7336,  34116,    896,
         9402,  28879,   5936,   7385,  36955,    456,   2083,   9113,
         2018,  19080,  12896,  33196,  10959,   4017,  12790,  30809,
         1424,    769,  27090,    664,     12,    591,   2352,   5571,
          901,   1870,  17758,    997,   5912,   2880,   9958,  25231,
          757,   1005,   1214,   5000,   2317,   1412,  80831,    672,
      

In [16]:
#What do the reporesentative points look like?
X_representative = sme.df.iloc[representative_idx]
X_representative

,RecordId,Gender,Distance,YearsWorked,TrainingHours,WorkLifeBalance,NumOfProjects,JobInvolvement,TeamSize,MentorshipReceived,...,PreviousSalary,Salary,SelfReview,SupervisorReview,Left,Raise,ReviewDiff,ReviewOverRaise,LongDist,HighStress
10976,11076,0,~10miles,6,72,1,13,4,12,0,...,56000,57000,5.0,5.0,1,0.017857,0.0,278.440732,0,1
743,843,1,~10miles,10,58,4,5,5,2,1,...,70000,71000,4.0,4.0,0,0.014286,0.0,278.053625,0,0
2789,2889,0,~10miles,5,41,5,9,2,14,0,...,53000,55000,4.0,4.0,0,0.037736,0.0,105.719842,0,0
7042,7142,0,~20miles,9,68,2,11,3,12,1,...,52000,54000,5.0,4.0,1,0.038462,1.0,129.662877,1,0
8579,8679,0,~15miles,4,13,3,1,5,10,1,...,79000,79000,5.0,5.0,1,0.000000,0.0,50000.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98720,98820,0,~20miles,6,18,1,4,2,16,0,...,82000,85000,3.0,2.0,0,0.036585,1.0,81.776478,1,0
317360,317460,1,~20miles,8,54,4,18,1,3,1,...,67000,70000,3.0,2.0,0,0.044776,1.0,66.850700,1,0
5322,5422,1,<5mile,6,39,3,6,4,6,0,...,59000,60000,3.0,5.0,0,0.016949,-2.0,175.961825,0,0
289,389,0,~10miles,9,70,5,12,1,18,0,...,51000,52000,4.0,5.0,1,0.019608,-1.0,202.964879,0,1


In [17]:
#Look up the leave/stay value of the 500 representative points
y_representative= []
for i in representative_idx:
    y_representative.append(sme.ask({'RecordId': i+100})) #All record Id's are 100 more than the index value

In [18]:
#Propagate the label value to each point in the cluster.
y_train_propagated = np.empty(len(sme.df), dtype=np.int32)
for i in range(n_clusters):
    y_train_propagated[kmeans.labels_ == i] = y_representative[i]

In [19]:
#Assign the labels and leave/stay values to the DF
sme.df['Labels'] = kmeans.labels_
sme.df['y_pred'] = y_train_propagated

In [20]:
sme.df['ClusteringOff']= abs(sme.df['y_pred'] - sme.df['Left']) #Which points are incorrectly labeled / is the clustering off for?

In [21]:
#What Percent incorrect is each cluster?
percent_df = sme.df.groupby('Labels')['ClusteringOff'].sum()/sme.df.groupby('Labels')['ClusteringOff'].count()
percent_df
#percent_df[percent_df>.5].count()

Labels
0      0.054398
1      0.071371
2      0.130876
3      0.446788
4      0.622296
         ...   
495    0.453642
496    0.123596
497    0.071782
498    0.049571
499    0.062319
Name: ClusteringOff, Length: 500, dtype: float64

In [22]:
#What is the total accuracy of the model?
1- sme.df['ClusteringOff'].sum() / sme.df['ClusteringOff'].count()

0.7440359999999999

In [23]:
#What percetange of "Leaving" predictions did our model get wrong?
sme.df[sme.df['Left']==1]['ClusteringOff'].sum() / sme.df[sme.df['Left']==1]['ClusteringOff'].count()

0.3681238605571287

In [24]:
#What percetange of "Staying" predictions did our model get wrong?
sme.df[sme.df['Left']==0]['ClusteringOff'].sum() / sme.df[sme.df['Left']==0]['ClusteringOff'].count()

0.19262124498105082

In [25]:
#How many "Stays" should be "leaves?
sme.df[sme.df['Left']==1]['ClusteringOff'].sum()

66432

In [26]:
#How many "leaves" Should be "stays"?
sme.df[sme.df['Left']==0]['ClusteringOff'].sum()

61550